# AUTETA model implementation in pyomo


https://www.pep-net.org/sites/pep-net.org/files/AUTETA-en.pdf

## 1 Preparation

### 1.1 Load libraries

In [ ]:
import pyomo.environ as pyo
from pyomo.environ import AbstractModel, Set, Param, Var, Constraint
import pyomo.dataportal.DataPortal as DataPortal
import pandas as pd
import json
from pyomo.core.expr.numvalue import value as constraintValue
import idaes.core.util as idaes_utils
from pyomo.core.expr.current import identify_variables
from pyomo.common.collections import ComponentSet
import numpy as np
from openpyxl import load_workbook

### 1.2 Rewrite data

In [ ]:
df = pd.read_excel('data/AUTETA.xlsx', sheet_name='SAM', header=[0,1], index_col=[0,1])
df.to_json('data/AUTETA.json')

### 1.3 Helper functions

In [ ]:
sam_dict = {}

## Make some helper functions
## Dictionary is organizec as "FROM (C, D) TO (A, B)"
## But commands will come in form (TO (A, B) FROM (C, D))
def sam(c, over=[], prices=None):
    
    coord = [0,0,0,0]
    d = {}
    p = {}
    if prices:
        if isinstance(prices, int):
            if(len(over) == 1):
                for cat in over[0]:
                    p[cat] = prices
            elif(len(over) == 2):
                for cat in over[1]:
                    p[cat] = prices
            elif (len(over) == 0):
                p['singular'] = prices
        else:
            p = prices
    p['ADM'] = 1 #hack
    
    # print(p)
    ## need to loop over levels:
    if (len(over) == 0):
        # print(sam_dict[f"('{c[0]}', '{c[1]}')"][f"('{c[2]}', '{c[3]}')"])
        if ('singular' in list(p.keys())):
            return sam_dict[f"('{c[2]}', '{c[3]}')"][f"('{c[0]}', '{c[1]}')"] / p['singular']    
        else:
            return sam_dict[f"('{c[2]}', '{c[3]}')"][f"('{c[0]}', '{c[1]}')"]
        # return sam[(c[0],c[1])][(c[2],c[3])]
        ## return the coordinate
    if len(over) == 1:
        idx = c.index(0)
        for cat in over[0]:
            coord = c.copy()
            coord[idx] = cat
            d[cat] = sam(coord)
            if d[cat] is None: d[cat] = 0 #fix nonetype values
            ## adjust for prices
            if cat in p.keys():
                d[cat] = d[cat]/p[cat]
    if len(over) == 2:
        idx = c.index(0)
        idx2 = c.index(1)
        for cat in over[0]:
            for cat2 in over[1]:
                coord = c.copy()
                coord[idx] = cat
                coord[idx2] = cat2
                d[(cat,cat2)] = sam(coord)
                if d[(cat,cat2)] is None: d[(cat,cat2)] = 0 #fix nonetype values
                ## assume only 2nd variable is adjusted for prices
                if cat in p.keys():
                    d[(cat,cat2)] = d[(cat,cat2)]/p[cat]
                
    


    # print(d)
    return d

## helper function for retriving initial values
def val(obj, sub1=None, sub2=None, default=None):
    try:
        if isinstance(obj, Var):
            if (sub2): return obj._value_init_value[(sub1,sub2)]
            if (sub1): return obj._value_init_value[sub1]
            return obj._value_init_value
        if isinstance(obj, Param):
            values = obj.default()
            if (sub2): return values[(sub1,sub2)]
            if (sub1): return values[sub1]
            return values
    except KeyError:
        if default is not None:
            return default
        else:
            raise


def divide(d1, d2):
    d = {}
    for key in d1.keys():
        d[key] = d1[key]/d2[key]
    return d


### 1.4 Initialize model

In [ ]:
m = AbstractModel()

## 2 Define sets

In [ ]:
sectors = ['AGR','MAN','SER','PUB']
tradeables = ['AGR','MAN','SER']
goods = ['AGR','MAN']
households = ['SAL', 'CAP']
m.sectors = Set(dimen=1,initialize=sectors, doc='Industries and commodities') 
m.tradeables = Set(dimen=1, initialize=tradeables, doc='Tradeable commodities')
m.goods = Set(dimen=1, initialize=goods, doc='Goods')
m.households = Set(dimen=1, initialize=households, doc='Households')

## 3 Load data

In [ ]:
sam_dict = DataPortal()
sam_dict.load(filename='data/AUTETA.json', encoding='utf-8')
sam_dict.data()

## 4. Variables and Parameters

### 4.1 Variables

In [ ]:
#note structure: TO (AG, CAP) FROM (AG, FIRM) 

# Prices (base)
prices = {
    'P': {
        'AGR': 1,
        'MAN': 1,
        'SER': 1,
        'PUB': 1
    },
    'R': {
        'AGR': 1,
        'MAN': 1,
        'SER': 1
    },
    'W': 1  
}
m.P       = pyo.Var(m.sectors, doc='Price of commodity i', initialize=prices['P'])
m.R       = pyo.Var(m.tradeables, doc='Rental rate of capital in industry j', initialize=prices['R'])
m.W       = pyo.Var(doc='Wage rate', initialize=prices['W'])

## ACCOUNTING MATRIX VARIABLES
m.DIV     = Var(doc='Dividends', initialize=sam(['AG','CAP','AG','FIRM']))
m.IT      = Var(doc='Total investment', initialize=sam(['OTH', 'TOT','OTH','ACC']))
m.SF      = Var(doc='Business savings', initialize=sam(['OTH','ACC','AG','FIRM']))
m.SH      = Var(m.households, doc='Savings of type h households', initialize=sam(['OTH', 'ACC','AG', 0], over=[households]))
m.YF      = Var(doc='Business income', initialize=sam(['OTH','TOT','AG','FIRM']))
m.YH      = Var(m.households, doc='Income of type h households', initialize=sam(['OTH','TOT','AG',0], over=[households]))
m.G       = Var(doc='Current public expenditures', initialize=sam(['I','PUB','AG','GVT']))
m.TG      = Var(doc='Public transfers to salaried households', initialize=sam(['AG','SAL','AG','GVT']))
m.DTH     = Var(m.households, doc='Receipts from direct taxation on household h income', initialize=sam(['AG','GVT','AG',0], over=[households]))
m.DTF     = Var(doc='Receipts from direct taxation on firms\' income', initialize=sam(['AG','GVT','AG','FIRM']))
m.TI      = Var(m.tradeables, doc='Receipts from indirect tax on commodity tr', initialize=sam(['AG','GVT','I',0], over=[tradeables]))
m.YG      = Var(doc='Government income', initialize=sam(['OTH','TOT','AG','GVT']))
m.SG      = Var(doc='Government savings', initialize=sam(['OTH','ACC','AG','GVT']))



## Variables which need to be converted to volumes using base prices
m.LD      = Var(m.sectors, doc='Industry j demand for labour', initialize=sam(['F','LD','J',0], over=[sectors], prices=prices['W']))
m.KD      = Var(m.tradeables, doc='Industry tr demand for capital', initialize=sam(['F','KD','J',0], over=[tradeables], prices=prices['R']))
m.XS      = Var(m.sectors, doc='Output of industry j', initialize=sam(['OTH','TOT','J',0,], over=[sectors], prices=prices['P']))


# Get tax rate parameter and price including taxes
tx = dict(map(lambda x: (x, m.TI._value_init_value[x] / (prices['P'][x] * m.XS._value_init_value[x])), tradeables))
m.tx      = Param(m.tradeables, doc='Tax rate on commodity tr', initialize=tx, mutable=True)
m.PD      = Var(m.tradeables, doc='Price of commodity tr (including taxes)', 
            initialize=dict(map(lambda tr: (tr, (1+tx[tr])/prices['P'][tr]), tradeables)))

## Variables which need to be converted to volumes using prices including taxes
m.C       = Var(m.tradeables, m.households, doc='Consumption of commodity i by type h households', 
                initialize=sam(['I',0,'AG',1], over=[tradeables, households], prices=val(m.PD)))
m.INV     = Var(m.tradeables, doc='Final demand of commodity i for investment purposes', 
                initialize=sam(['I',0,'OTH','ACC'], over=[tradeables], prices=val(m.PD)))
m.DI      = Var(m.tradeables, m.sectors, doc='Intermediate consumption of commodity tr in industry j', 
                initialize=sam(['I',0,'J',1], over=[tradeables, sectors], prices=val(m.PD)))

# Other variables
m.LS      = Var(doc='Total labour supply', initialize=sum(m.LD._value_init_value.values()))
m.KS      = Var(m.tradeables, doc='Capital supply in industry tr', initialize=m.KD._value_init_value)

#Value-added is tricky because there's a special rules for the public sector
VA = dict(map(lambda tr: (tr, m.LD._value_init_value[tr] + m.KD._value_init_value[tr]), tradeables))
PVA = dict(map(lambda tr: (tr, (prices['W']*m.LD._value_init_value[tr] + prices['R'][tr]*m.KD._value_init_value[tr])/VA[tr]), tradeables))
VA['PUB'] = m.LD._value_init_value['PUB']
PVA['PUB'] = prices['W']
m.VA      = Var(m.sectors, doc='Value added of industry j', initialize=VA)
m.PVA     = Var(m.sectors, doc='Price of value added of industry j', initialize=PVA)

# variables summed across other variables
m.DIT     = Var(m.tradeables, doc='Total intermediate demand of commodity tr', 
                initialize=dict(map(lambda tr: (tr, sum(val(m.DI,tr,j) for j in sectors)), tradeables)))
m.CI      = Var(m.sectors, doc='Total intermediate consumption of industry j', 
                    initialize=dict(map(lambda j: (j, sum(val(m.DI,tr,j) for tr in tradeables)), sectors)))
m.PCI     = Var(m.sectors, doc='Intermediate consumption price index of industry j',
                initialize=dict(map(lambda j: (j, sum(val(m.PD, tr)*val(m.DI,tr,j) for tr in tradeables) / m.CI._value_init_value[j]), sectors)))
m.YDH     = Var(m.households, doc='Disposable income of type h households', 
                initialize=dict(map(lambda h: (h, m.YH._value_init_value[h] - m.DTH._value_init_value[h]), households)))
m.CTH     = Var(m.households, doc='Consumption budget of type h households',
                initialize=dict(map(lambda h: (h, m.YDH._value_init_value[h] - m.SH._value_init_value[h]), households)))

## other variables 
m.LEON    = Var(doc='Excess supply on the market for services', initialize=0)

### 4.2 Parameters

In [ ]:
alpha       = dict(map(lambda tr: (tr, 
                        val(m.W)*val(m.LD, tr) / (val(m.PVA, tr)*val(m.VA, tr))), 
                        tradeables))
m.alpha     = Param(m.tradeables, doc='Elasticity (Cobb-Douglas production function)', 
                        initialize=alpha)
m.A         = Param(m.tradeables, doc='Scale parameter (Cobb-Douglas - Production function)', 
                        initialize=dict(map(lambda tr: (tr, 
                        val(m.VA, tr)/(val(m.LD, tr)**alpha[tr]*val(m.KD, tr)**(1-alpha[tr]))), 
                        tradeables)))
m.aij       = Param(m.tradeables, m.sectors, doc='Coefficient (Leontief - intermediate consumption)', 
                    initialize=dict([((tr, j), val(m.DI, tr, j)/val(m.CI, j)) for tr in tradeables for j in sectors]))
m.gamma     = Param(m.tradeables, m.households, doc='Share of commodity i in type h household consumption budget', 
                    initialize=dict([((tr, h), val(m.PD, tr)*val(m.C, tr, h)/val(m.CTH, h)) for tr in tradeables for h in households]))
m.io        = pyo.Param(m.sectors, doc='Coefficient (Leontief - total intermediate consumption)', 
                        initialize=dict(map(lambda j: (j, 
                        val(m.CI, j)/val(m.XS, j)), 
                        sectors)))
m.lmbda     = pyo.Param(doc='Share of capital income receied by capitalists', 
                        initialize=(val(m.YH, 'CAP')-m.DIV)/sum(val(m.R, tr)*val(m.KD, tr) for tr in tradeables))
m.mu        = pyo.Param(m.sectors, doc='Share of commodity i in total investment expenditures', 
                        initialize=dict(map(lambda tr: (tr, 
                        val(m.PD, tr)*val(m.INV, tr)/val(m.IT)), 
                        tradeables)))
m.psi       = pyo.Param(m.households, doc='Average propensity to save of type h household', 
                        initialize=dict(map(lambda h: (h, 
                        val(m.SH, h)/val(m.YDH, h)), 
                        households)))
m.v         = pyo.Param(m.sectors, doc='Coefficient (Leontief - value added)', 
                        initialize=dict(map(lambda j: (j, 
                        val(m.VA, j)/val(m.XS, j)), 
                        sectors)))
m.tyh       = pyo.Param(m.households, doc='Direct tax rate on household h income', 
                        initialize=dict(map(lambda h: (h, 
                        val(m.DTH, h)/val(m.YH, h)), 
                        households)))
m.tyf       = pyo.Param(doc='Direct tax on rate of firms\' income', 
                        initialize=val(m.DTF)/val(m.YF))

## 5 Equations

In [ ]:
## Production
def XSEQ(m, j):
    return m.VA[j] == m.v[j]*m.XS[j]
m.XSEQ = pyo.Constraint(m.sectors, rule=XSEQ, doc='Value added demand in industry j (Leontief)')

def CIEQ(m, j):
    return m.CI[j] == m.io[j]*m.XS[j]
m.CIEQ = pyo.Constraint(m.sectors, rule=CIEQ, doc='Total intermediate consumption demand in industry j (Leontief)')

def VAEQ(m, tr):
    return m.VA[tr] == m.A[tr]*(m.LD[tr]**m.alpha[tr])*(m.KD[tr]**(1-m.alpha[tr]))
m.VAEQ = pyo.Constraint(m.tradeables, rule=VAEQ, doc='Cobb-Douglas between labour and capital')

def LDEQ(m, tr):
    return m.W*m.LD[tr] == m.alpha[tr]*m.PVA[tr]*m.VA[tr]
m.LDEQ = pyo.Constraint(m.tradeables, rule=LDEQ, doc='Demand for labour by industry j')

def LDPEQ(m):
    return m.LD['PUB'] == m.VA['PUB']
m.LDPEQ = pyo.Constraint(rule=LDPEQ, doc='Demand for labour in the public sector')

def KDEQ(m, tr):
    return m.R[tr]*m.KD[tr] == (1-m.alpha[tr])*m.PVA[tr]*m.VA[tr]
m.KDEQ = pyo.Constraint(m.tradeables, rule=KDEQ, doc='Demand for capital by industry j')

def DIEQ(m, tr, j):
    return m.DI[(tr, j)] == m.aij[(tr,j)]*m.CI[j]
m.DIEQ = pyo.Constraint(m.tradeables, m.sectors, rule=DIEQ, doc='Intermediate consumption of commodity i by sector j')


In [ ]:
## Income and savings
def YHSEQ(m):
    return m.YH['SAL'] == m.W*sum(m.LD[j] for j in m.sectors) + m.TG
m.YHSEQ = Constraint(rule=YHSEQ, doc='Household income (workers)')

def YHCEQ(m):
    return m.YH['CAP'] == m.lmbda*sum(m.R[tr]*m.KD[tr] for tr in m.tradeables) + m.DIV
m.YHCEQ = Constraint(rule=YHCEQ, doc='Household income (workers)')

def YDHEQ(m, h):
    return m.YDH[h] == m.YH[h] - m.DTH[h]
m.YDHEQ = Constraint(m.households, rule=YDHEQ, doc='Household disposable income')

def SHEQ(m, h):
    return m.SH[h] == m.psi[h]*m.YDH[h]
m.SHEQ = Constraint(m.households, rule=SHEQ, doc='Household h savings')

def CTHEQ(m, h):
    return m.CTH[h] == m.YDH[h] - m.SH[h]
m.CTHEQ = Constraint(m.households, rule=CTHEQ, doc='Consumption budget for household h')

def YFEQ(m):
    return m.YF == (1-m.lmbda)*sum(m.R[tr]*m.KD[tr] for tr in m.tradeables) 
m.YFEQ = Constraint(rule=YFEQ, doc='Firms income')

def SFEQ(m):
    return m.SF == m.YF - m.DIV - m.DTF
m.SFEQ = Constraint(rule=SFEQ, doc='Firms savings')

def YGEQ(m):
    return m.YG == sum(m.TI[tr] for tr in m.tradeables) + sum(m.DTH[h] for h in m.households) + m.DTF
m.YGEQ = Constraint(rule=YGEQ, doc='Government income')

def TIEQ(m, tr):
    return m.TI[tr] == m.tx[tr]*m.P[tr]*m.XS[tr]
m.TIEQ = Constraint(m.tradeables, rule=TIEQ, doc='Receipts from indirect taxation')

def DTHEQ(m, h):
    return m.DTH[h] == m.tyh[h]*m.YH[h]
m.DTHEQ = Constraint(m.households, rule=DTHEQ, doc='Receipts from income taxes (households)')

def DTFEQ(m):
    return m.DTF == m.tyf*m.YF
m.DTFEQ = Constraint(rule=DTFEQ, doc='Receipts from income taxes (firms)')

def SGEQ(m):
    return m.SG == m.YG - m.G - m.TG
m.SGEQ = Constraint(rule=SGEQ, doc='Government savings')



In [ ]:
## Demand
def CEQ(m, tr, h):
    return m.PD[tr]*m.C[(tr,h)] == m.gamma[(tr,h)]*m.CTH[h]
m.CEQ = Constraint(m.tradeables, m.households, rule=CEQ, doc='Household h consumption of commodity tr')

def INVEQ(m, tr):
    return m.PD[tr]*m.INV[tr] == m.mu[tr]*m.IT
m.INVEQ = Constraint(m.tradeables, rule=INVEQ, doc='Investment in commodity tr')

def DITEQ(m, tr):
    return m.DIT[tr] == sum(m.DI[(tr,j)] for j in m.sectors)
m.DITEQ = Constraint(m.tradeables, rule=DITEQ, doc='Intermediate demand for commodity i')

In [ ]:
## Prices
def PVAPEQ(m):
    return m.PVA['PUB'] == m.W
m.PVAPEQ = Constraint(rule=PVAPEQ, doc='Equivalence between PVA and W for public sector')

def PCIEQ(m, j):
    return m.PCI[j]*m.CI[j] == sum(m.PD[tr]*m.DI[(tr,j)] for tr in tradeables)
m.PCIEQ = Constraint(m.sectors, rule=PCIEQ, doc='Intermediate consumption price index')

def CPEQ(m, j):
    return m.P[j]*m.XS[j] == m.PVA[j]*m.VA[j] + m.PCI[j]*m.CI[j]
m.CPEQ = Constraint(m.sectors, rule=CPEQ, doc='Production costs for sector j')

def PDEQ(m, tr):
    return m.PD[tr] == m.P[tr]*(1+m.tx[tr])
m.PDEQ = Constraint(m.tradeables, rule=PDEQ, doc='Price of commodity tr including taxes')

In [ ]:
## Equilibrium
def PEQ(m, bns):
    return m.XS[bns] == sum(m.C[(bns, h)] for h in m.households) + m.DIT[bns] + m.INV[bns]
m.PEQ = Constraint(m.goods, rule=PEQ, doc='Domestic absorption (over goods)')

def PPUBEQ(m):
    return m.G == m.P['PUB']*m.XS['PUB']
m.PPUBEQ = Constraint(rule=PPUBEQ, doc='Equilibrium on the market for public services')

def WEQ(m):
    return m.LS == sum(m.LD[j] for j in m.sectors)
m.WEQ = Constraint(rule=WEQ, doc='Labour market equilibrium')

def REQ(m, tr):
    return m.KS[tr] == m.KD[tr]
m.REQ = Constraint(m.tradeables, rule=REQ, doc='Capital market equilibrium')

def ITEQ(m):
    return m.IT == sum(m.SH[h] for h in m.households) + m.SF + m.SG
m.ITEQ = Constraint(rule=ITEQ, doc='Investment-savings equilibrium')

## Other
def WALRAS(m):
    return m.LEON == m.XS['SER'] - sum(m.C[('SER', h)] for h in m.households) - m.DIT['SER'] - m.INV['SER']
m.WALRAS =  Constraint(rule=WALRAS, doc="Verification of Walras' law")


## 6 Create instances

### 6.1 Helper functions

In [ ]:
def count_nonfixed(i, active=True):
    
    num_non_fixed = 0
    num_fixed = 0
    num_vars = 0
    for block in i.block_data_objects(active=active):
        var_set = ComponentSet()
        for c in block.component_data_objects(
                ctype=Constraint, active=True, descend_into=True):
            for v in identify_variables(c.body):
                var_set.add(v)

        for v in var_set:
            if v.is_fixed():
                num_fixed += 1
            else: 
                num_non_fixed += 1
        num_vars = len(var_set)
 
    return num_non_fixed, num_fixed, num_vars

def count_constraints(i, active=True):
    num_constraints = 0
    for block in i.block_data_objects(active=active):
        for data in block.component_map(pyo.Constraint, active=active).values():
            for key in data._data.keys():
                num_constraints += 1
    return num_constraints

def checkSquareness(inst):
    num_non_fixed, num_fixed, num_vars = count_nonfixed(inst)
    if (num_non_fixed == inst.nconstraints()):
        print('Instance IS SQUARE.')
    else:
        print('Instance is NOT SQUARE!')

def checkFeasibility(inst, limit=1e-9):
    feasible = True
    num_checked = 0
    for block in inst.block_data_objects(active=True):
        for constraint in block.component_map(pyo.Constraint, active=True).values():
            # print(constraint, len(list(constraint._data.keys())), constraint.doc)
            for index in constraint.index_set():
                if index in list(constraint._data.keys()):
                    num_checked = num_checked + 1
                    if abs(constraintValue(constraint[index])) > limit:
                        feasible = False
                        print('Infeasibility:', constraint, index, constraintValue(constraint[index]))
    print(f'Checked {num_checked} constraints.')
    
    if feasible:
        print('All equations balanced. Instance is feasible')

def has_value(obj, key):
    if obj[key].value != 0:
         if obj[key].value != 0.0:
             if obj[key].value is not None:
                 return True
    return False

In [ ]:
def applyFixes(inst, m):
    inst.P['AGR'].fix() #1 numeraire
    inst.KS.fix()       #3
    inst.LS.fix()       #1
    inst.DIV.fix()      #1
    inst.G.fix()      #1
    inst.TG.fix()      #1

### 6.2 Define optimization objective

In [ ]:
def obj_expression(m):
    return sum(m.C[(tr, h)] for tr,h in m.tradeables*m.households)

m.OBJ = pyo.Objective(rule=obj_expression, sense=pyo.maximize)

### 6.3 Create base instance

In [ ]:
inst_base = m.create_instance()
applyFixes(inst_base, m)

In [ ]:
for block in inst_base.block_data_objects(active=True):
    print('variables:          ', idaes_utils.model_statistics.number_variables_in_activated_equalities(block))
    print('  fixed:            ', idaes_utils.model_statistics.number_fixed_variables_in_activated_equalities(block))
    print('  unfixed:          ', idaes_utils.model_statistics.number_unfixed_variables_in_activated_equalities(block))
    print('constraints:        ', idaes_utils.model_statistics.number_activated_equalities(block))
    print('degrees of freedom: ', idaes_utils.model_statistics.degrees_of_freedom(block))
    checkSquareness(block)
    checkFeasibility(block)
    idaes_utils.model_statistics.report_statistics(block)


In [ ]:
# Display some of the parameters
inst_base.A.display()
inst_base.alpha.display()
inst_base.io.display()
inst_base.v.display()
inst_base.aij.display()
inst_base.gamma.display()
inst_base.psi.display()
inst_base.mu.display()
inst_base.lmbda.display()

### 6.4 Create scenarios

In [ ]:
### 25% reduction in indirect taxation:
inst_scenario_1 = m.create_instance()
applyFixes(inst_scenario_1, m)
for tr in tradeables:
    inst_scenario_1.tx[tr] = 0.75 * inst_scenario_1.tx[tr]


# Capital shock on service sector
inst_scenario_2 = m.create_instance()
applyFixes(inst_scenario_2, m)
inst_scenario_2.KS['SER'] = 1.1 * inst_scenario_2.KS['SER'].value

# Shock to numeraire
inst_scenario_3 = m.create_instance()
applyFixes(inst_scenario_3, m)
inst_scenario_3.P['AGR'] = 1.5 * inst_scenario_3.P['AGR'].value
inst_scenario_3.DIV = 1.5 * inst_scenario_3.DIV.value
inst_scenario_3.G = 1.5 * inst_scenario_3.G.value
inst_scenario_3.TG = 1.5 * inst_scenario_3.TG.value


## 7 Solve instances

In [ ]:
opt = pyo.SolverFactory('ipopt')
opt.options['max_cpu_time'] = 120 #seconds
opt.options['warm_start_init_point'] = 'yes'
opt.options['halt_on_ampl_error'] = 'yes'

In [ ]:
res_base = opt.solve(inst_base, tee=True)
if (res_base.Solver.Status != 'ok'):
    print('Status NOT ok!')
if (res_base.Solver[0]['Termination condition'] != 'optimal'):
    print('Optimal solution NOT found!')
else:
    print('Optimal solution found!')
res_base

In [ ]:
res_scenario_1 = opt.solve(inst_scenario_1, tee=False)
if (res_scenario_1.Solver.Status != 'ok'):
    print('Status NOT ok!')
if (res_scenario_1.Solver[0]['Termination condition'] != 'optimal'):
    print('Optimal solution NOT found!')
else:
    print('Optimal solution found!')
res_scenario_1

In [ ]:
res_scenario_2 = opt.solve(inst_scenario_2, tee=False)
if (res_scenario_2.Solver.Status != 'ok'):
    print('Status NOT ok!')
if (res_scenario_2.Solver[0]['Termination condition'] != 'optimal'):
    print('Optimal solution NOT found!')
else:
    print('Optimal solution found!')
res_scenario_2

In [ ]:
res_scenario_3 = opt.solve(inst_scenario_3, tee=False)
if (res_scenario_3.Solver.Status != 'ok'):
    print('Status NOT ok!')
if (res_scenario_3.Solver[0]['Termination condition'] != 'optimal'):
    print('Optimal solution NOT found!')
else:
    print('Optimal solution found!')
res_scenario_3

## 8 Compare solutions

### 8.1 Helper functions

In [ ]:
def compare_scenarios(instances_dict):
    results_obj = {}    
    for instance_key in instances_dict.keys():
        for block in instances_dict[instance_key].block_data_objects(active=True):
            var_set = ComponentSet()
            for v in block.component_map(pyo.Var, active=True).values():
                if not v._name in results_obj.keys():
                    results_obj[v._name] = {}
                for key in v.keys():
                    if not key in results_obj[v._name].keys():
                        results_obj[v._name][key] = {}
                    results_obj[v._name][key][instance_key] = v[key].value

    return results_obj

def results_dict_to_pandas_dict(results_obj):
    pandas_dict = {}
    for key in results_obj.keys():
        df = pd.DataFrame.from_dict(results_obj[key], orient='index')
        columns = list(df.columns)
        for column in columns:
            if column == columns[0]:
                continue
            df[f'{column} %diff'] = df[column]/df[columns[0]] - 1
            df[f'{column} %diff'] = np.where((df[column] == 0) & (df[columns[0]] == 0), 
                                             0.0, df[f'{column} %diff'])
        pandas_dict[key] = df
    return pandas_dict

def write_pandas_dict_to_file(pandas_dict, wb_path):
    # book = load_workbook(wb_path)
    writer = pd.ExcelWriter(wb_path)
    workbook = writer.book
    percent_fmt = workbook.add_format({'num_format': '0.00%'})
    wrapped_text = workbook.add_format({
        'text_wrap': True
    })
    
    # writer.book = book
    for key in pandas_dict.keys():
        df = pandas_dict[key]
        first_row = 2
        last_row = 1 + len(df)
        num_index_cols = 1
        if type(df.index) == pd.core.indexes.multi.MultiIndex:
            num_index_cols = len(df.index[0])
        base_col = num_index_cols + 1
        num_scenarios = int((len(list(df.columns)) - 1) / 2)
        
        df.to_excel(writer, sheet_name=key, index=True)
        
        workbook = writer.book
        worksheet = writer.sheets[key]
        
        # Apply a conditional format to the cell range.
        worksheet.conditional_format(
            f'{chr(64 + base_col + num_scenarios + 1)}{first_row}:{chr(64 + base_col + num_scenarios*2)}{last_row}', 
            {'type': '3_color_scale'})
        worksheet.set_column(f'{chr(64 + base_col + num_scenarios + 1)}:{chr(64 + base_col + num_scenarios*2)}', None, percent_fmt)
        worksheet.set_row(0, 30, wrapped_text)
        
    writer.save()
    print('done.')

### 8.2 Store comparisons

In [ ]:
results_obj = compare_scenarios({
    'base': inst_base, 
    'scenario 1': inst_scenario_1,
    'scenario 2': inst_scenario_2,
    'scenario 3': inst_scenario_3
    })
    


In [ ]:
pandas_dict = results_dict_to_pandas_dict(results_obj)

In [ ]:
write_pandas_dict_to_file(pandas_dict, 'output/results_pyomo.xlsx')